<a href="https://colab.research.google.com/github/Nandini37/Python_ML_AI/blob/main/Poisonous_Mushroom_EDA%2C_feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'playground-series-s4e8:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F76727%2F9045607%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240903%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240903T123914Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4e46daa0cc66d23f0a47602d72716eb21a168f20b85b4013275fdc04d0c91b226899ffce5dfb0950cc5a39b54d8e7169913c1ba92446191ed7c75975a1e3fe159d650a8ce29b42693d7491914c1073a737dea04035930d19d3be14c68b01baeebee43ee62667cda56e3bee5c4a26cae47cb63f83c258e53ddc6046ac67c6faa0b2e48f12bc0d15c309897f6782f0f93e82dd3d6f87b41d2b82a5a60de4acbf0629e94e8eba43d0c33055f250ae5f9d6f8477665ad8e110fb3c5a24b3c4399fe48607383e214aa36fe1df553f3b989bf8951b0eb0e140d0833a267bd3192618d1df00a5f177fd11cc608343584f39c61355f03b87028b042f2e00951749f50930'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 86301661 bytes downloaded
Downloaded and uncompressed: playground-series-s4e8
Data source import complete.


In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e8/test.csv
/kaggle/input/playground-series-s4e8/train.csv
/kaggle/input/playground-series-s4e8/sample_submission.csv


In [7]:
import pandas as pd
import numpy as np
import seaborn as sns


pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [49]:
df = pd.read_csv("/kaggle/input/playground-series-s4e8/train.csv")

In [50]:
df.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,4.51,15.39,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,4.79,6.48,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,6.85,9.93,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,4.16,6.53,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,3.37,8.36,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


In [51]:
df.shape

(3116945, 22)

In [52]:
df.drop_duplicates()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,4.51,15.39,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,4.79,6.48,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,6.85,9.93,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,4.16,6.53,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,3.37,8.36,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116940,3116940,e,9.29,f,NaN,n,t,NaN,NaN,w,12.14,18.81,b,NaN,w,u,w,t,g,NaN,d,u
3116941,3116941,e,10.88,s,NaN,w,t,d,c,p,6.65,26.97,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
3116942,3116942,p,7.82,x,e,e,f,a,NaN,w,9.51,11.06,NaN,NaN,y,NaN,w,t,z,NaN,d,a
3116943,3116943,e,9.45,p,i,n,t,e,NaN,p,9.13,17.77,NaN,y,w,NaN,NaN,t,p,NaN,d,u


In [54]:
df.isnull().sum()/df.shape[0]

,0
id,0.000000
class,0.000000
cap-diameter,0.000001
cap-shape,0.000013
cap-surface,0.215282
cap-color,0.000004
does-bruise-or-bleed,0.000003
gill-attachment,0.168093
gill-spacing,0.403740
gill-color,0.000018


In [55]:
df.columns

Index(['id', 'class', 'cap-diameter', 'cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-height', 'stem-width', 'stem-root', 'stem-surface', 'stem-color',
       'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color',
       'habitat', 'season'],
      dtype='object')

In [56]:
df.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,4.51,15.39,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,4.79,6.48,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,6.85,9.93,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,4.16,6.53,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,3.37,8.36,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


In [57]:

cols = df.columns.to_list()

for c in cols:
    print(c, df[c].nunique())

id 3116945
class 2
cap-diameter 3913
cap-shape 74
cap-surface 83
cap-color 78
does-bruise-or-bleed 26
gill-attachment 78
gill-spacing 48
gill-color 63
stem-height 2749
stem-width 5836
stem-root 38
stem-surface 60
stem-color 59
veil-type 22
veil-color 24
has-ring 23
ring-type 40
spore-print-color 32
habitat 52
season 4


In [58]:
col2 = []
for c in cols:
    if df[c].nunique() <= 30:
        col2.append(c)


print("featurelist",col2)
for c in col2:
    print(c, df[c].unique())




featurelist ['class', 'does-bruise-or-bleed', 'veil-type', 'veil-color', 'has-ring', 'season']
class ['e' 'p']
does-bruise-or-bleed ['f' 't' 'd' 'has-ring' 'w' 'o' 'b' 'x' 'p' nan 'g' 'y' 'r' 'a' 'l' 'i'
 'c' 'n' 'z' 's' 'k' 'h' '3.43' 'e' '4.42' '2.9' 'u']
veil-type [nan 'u' 'd' 'a' 'h' '21.11' 'g' 'c' 'e' 'y' 'i' 'f' 'is None' 't' 'w' 'p'
 'b' 's' 'k' 'r' 'l' 'n' '5.94']
veil-color [nan 'n' 'w' 'k' 'y' 'e' 'u' 'p' 'd' 'g' 'r' 'h' 's' '8.25' 't' 'c' 'o'
 'i' '2.49' 'f' 'a' 'b' 'l' 'z' '3.32']
has-ring ['f' 't' 'h' 'r' 'y' 'c' 'e' 'g' 'l' 's' nan 'p' 'x' 'k' 'z' 'f has-ring'
 'd' 'o' 'n' 'm' 'i' '10.3' 'w' 'a']
season ['a' 'w' 'u' 's']


In [64]:
# Replace 'nan' with actual NaN

for col in col2:
     df[col] = df[col].apply(lambda x: pd.NA if str(x).replace('.', '', 1).isdigit() else x)
df.replace('nan', pd.NA, inplace=True)


In [65]:
for c in col2:
    print(c, df[c].unique())

class ['e' 'p']
does-bruise-or-bleed ['f' 't' 'd' 'has-ring' 'w' 'o' 'b' 'x' 'p' nan 'g' 'y' 'r' 'a' 'l' 'i'
 'c' 'n' 'z' 's' 'k' 'h' <NA> 'e' 'u']
veil-type [nan 'u' 'd' 'a' 'h' <NA> 'g' 'c' 'e' 'y' 'i' 'f' 'is None' 't' 'w' 'p'
 'b' 's' 'k' 'r' 'l' 'n']
veil-color [nan 'n' 'w' 'k' 'y' 'e' 'u' 'p' 'd' 'g' 'r' 'h' 's' <NA> 't' 'c' 'o' 'i'
 'f' 'a' 'b' 'l' 'z']
has-ring ['f' 't' 'h' 'r' 'y' 'c' 'e' 'g' 'l' 's' nan 'p' 'x' 'k' 'z' 'f has-ring'
 'd' 'o' 'n' 'm' 'i' <NA> 'w' 'a']
season ['a' 'w' 'u' 's']


In [69]:
cols

['id',
 'class',
 'cap-diameter',
 'cap-shape',
 'cap-surface',
 'cap-color',
 'does-bruise-or-bleed',
 'gill-attachment',
 'gill-spacing',
 'gill-color',
 'stem-height',
 'stem-width',
 'stem-root',
 'stem-surface',
 'stem-color',
 'veil-type',
 'veil-color',
 'has-ring',
 'ring-type',
 'spore-print-color',
 'habitat',
 'season']

In [81]:
df['cap-shape'].value_counts()

,count
cap-shape,
x,1436026
f,676238
s,365146
b,318646
o,108835
p,106967
c,104520
d,65
e,60


In [84]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import numpy as np

# Assuming df is your DataFrame and 'stem-width' is the column of interest

# Step 1: Identify values that occur less than 1000 times
value_counts = df['stem-width'].value_counts()
values_to_replace = value_counts[value_counts < 1000].index

# Step 2: Replace these values with NaN
df['stem-width'] = df['stem-width'].replace(values_to_replace, np.nan)

# Step 3: Apply Iterative Imputer to impute NaN values
imputer = IterativeImputer()
df[['stem-width']] = imputer.fit_transform(df[['stem-width']])

# Now, 'stem-width' values that had less than 1000 occurrences are replaced with imputed values.


In [86]:
# import vaex
# import numpy as np

# # Load data into a Vaex DataFrame
# df = vaex.from_pandas(df)


# # Identify values to replace
# value_counts = df['stem-width'].value_counts()
# values_to_replace = value_counts[value_counts < 1000].index.tolist()

# # Replace values with NaN
# df['stem-width'] = df['stem-width'].apply(lambda x: np.nan if x in values_to_replace else x)

# # Export to Pandas for imputation if necessary, or handle imputation using custom Vaex methods
# df_pandas = df.to_pandas_df(columns=['stem-width'])

# # Apply Iterative Imputer in Pandas (if necessary)
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer

# imputer = IterativeImputer()
# df_pandas['stem-width'] = imputer.fit_transform(df_pandas[['stem-width']])

# # Convert back to Vaex DataFrame if needed
# #df_vaex = vaex.from_pandas(df_pandas)



,count
stem-width,
8.414042,854299
2.410000,7829
2.450000,7353
2.490000,7087
2.560000,6824
...,...
0.820000,1003
17.050000,1001
16.730000,1001


In [47]:


 # Impute missing values with mode for categorical data
imputer = SimpleImputer(strategy='most_frequent')
df[['veil-type', 'veil-color', 'does-bruise-or-bleed']] = imputer.fit_transform(df[['veil-type', 'veil-color', 'does-bruise-or-bleed']])



In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5, weights="uniform")  # Or "distance" for distance-weighted neighbors
df_imputed = imputer.fit_transform(df)

In [ ]:
# # Calculate the mean of 'does-bruise-or-bleed' for each group
# mean_values = df.groupby(['class', 'cap-shape', 'cap-surface', 'cap-color', 'habitat', 'season'])['does-bruise-or-bleed'].transform('mean')

# # Fill NaN values with the mean for their respective group
# df['does-bruise-or-bleed'] = df['does-bruise-or-bleed'].fillna(mean_values)